# Cuaderno 3: Métodos numéricos

En el cuaderno anterior aprendimos a usar **NumPy** para trabajar con matrices. Ahora vamos a dar un paso más: veremos cómo **aproximar integrales** y cómo resolver problemas donde queremos saber *cómo cambia una cantidad con el tiempo*.

Aunque gran parte de la neurociencia puede describirse con matemáticas, una buena parte de las matemáticas que se usan no puede resolverse exactamente. Esto puede parecer muy extraño: que uno pueda escribir algo en términos matemáticos y que no se pueda resolver de inmediato. Para sortear este problema usamos Métodos Numéricos para estimar la solución.

No te preocupes si nunca viste cálculo: la idea es entender con ejemplos sencillos, sin necesidad de fórmulas complicadas.

## Configuración

Primero, tenemos que preparar el cuaderno cargando algunas librerías de Python que nos van a ayudar a hacer cuentas y dibujar gráficos. Solo corré estas celdas una vez y ya queda listo el entorno.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets

### Funciones de graficado

In [ ]:
def plot_slope(dt):
  """
    Args:
      dt  : time-step
    Returns:
      A figure of an exponential, the slope of the exponential and the derivative exponential
  """

  t = np.arange(0, 5+0.1/2, 0.1)

  fig = plt.figure(figsize=(6, 4))
  # Exponential
  x = np.exp(0.3*t)
  plt.plot(t, x, label='x')
  # slope
  plt.plot([1, 1+dt], [np.exp(0.3*1), np.exp(0.3*(1+dt))],':og',label=r'$\frac{x(1+\Delta t)-x(1)}{\Delta t}$')
  # derivative
  plt.plot([1, 1+dt], [np.exp(0.3*1), np.exp(0.3*(1))+dt*0.3*np.exp(0.3*(1))],'-k',label=r'$\frac{dx}{dt}$')
  plt.legend()
  plt.plot(1+dt, np.exp(0.3*(1+dt)), 'og')
  plt.ylabel('x')
  plt.xlabel('t')
  plt.show()

def plot_euler(t, euler, analitica):
  # Creamos una fila con espacio para dos gráficas
  fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, layout='constrained')
  
  # Graficamos ambas soluciones para poder compararlas, la analítica en azul, y la discreta en rojo
  ax1.plot(t, analitica, "b")
  ax1.plot(t, euler, "r--")
  
  # Graficamos el error a la derecha
  ax2.plot(t, (xAna - xEuler) / xAna)
  
  # Anotamos los colores
  ax1.legend(['Solución analítica', 'Método de Euler'])
  
  # Anotamos los ejes
  ax1.set_xlabel("t")
  ax1.set_ylabel("x")
  ax2.set_xlabel("t")
  ax2.set_ylabel("error")
  
  plt.show()

## Método de Euler

Muchas veces queremos saber cómo una variable cambia con el tiempo. Por ejemplo, cómo se modifica la temperatura de un objeto, o el voltaje de una neurona.

El **método de Euler** es una técnica muy sencilla para estimar esos cambios paso a paso. La idea básica es usar la *pendiente* (qué tan rápido cambia algo) para aproximar el valor futuro.

Vamos a empezar viendo cómo se relaciona la pendiente con la idea de derivada.

Primero, pensemos en la **pendiente** de una curva: es qué tan inclinada está en un punto. En cálculo, a eso se le llama *derivada*.

In [ ]:
@widgets.interact(dt=(0.0, 4.0, 0.1))
def Pop_widget(dt):
  plot_slope(dt)

En vez de calcular derivadas exactas (lo cual puede ser complicado), vamos a aproximarlas con pasos pequeños.

### Ejemplo con constante de tiempo

Muchas dinámicas pueden escribirse con una fórmula como esta:

$$ \tau \dfrac{dx}{dt} = x $$

Esto significa que la variable $x$ cambia con el tiempo de manera proporcional a su propio valor. $\tau$ es una constante que marca qué tan rápido ocurre ese cambio. En realidad, podemos despejar la ecuación diferencial:

$$ \dfrac{dx}{dt} = \dot x = \dfrac{x}{\tau} $$

*Nota: Más adelante, en el código Python, usaremos la función `xdot()` para definir a $\dot{x}$*


¿Cómo podemos observar como cambia $x$ el tiempo y contestar, por ejempo, que valor toma $x$ en un instante de tiempo dado?

El método de Euler lleva el nombre de Leonhard Euler, quien lo presentó en su libro *Institutionum calculi integralis* (publicado en 1768). Este método se utiliza para aproximar cálculos de integrales de manera numérica. La idea principal es resolver el problema usando cálculos numéricos en lugar de fórmulas exactas.

Básicamente, consiste en aproximar la solución utilizando la siguiente fórmula (más abajo, en bonificación, encontrarás una explicación mas detallada):

$$x(t + \Delta t) = x(t) + \Delta t \dot x$$

Este método iterativo observa cómo cambia $x$ durante un intervalo de tiempo al sumar los cambios uno tras otro. Es el método más básico para realizar integraciones numéricas y se conoce como el *método de Euler*.

In [ ]:
# Condición inicial de x(t) en t=0, es decir, valor de x(0).
x0 = 1

# Valor de salto en el tiempo
dt = 0.1

# Constante temporal tau
tau = 1

# Tiempo máximo que queremos integrar
tmax = 3

# Definimos la funcion xdot
def xdot(x, t):
    return x / tau

# Periodo de tiempo que queremos observar
t = np.arange(0, tmax, dt) 

# Solución analítica
xAna = x0 * np.exp(t / tau)

# Solución discreta
x = np.array([x0])
for step in t:
    x = np.append(x, x[-1] + dt * xdot(x[-1], step))
xEuler = x[:-1]

# Imprimimos los resultados como una tabla
print("t\tAnalítica\tMétodo de Euler")
for i in range(len(t)):
    print(f"{round(t[i], 2)}\t{round(xAna[i], 2)}\t\t{round(xEuler[i], 2)}")

Para entender mejor los resultados, es muy útil **dibujarlos en un gráfico**. Así podemos comparar lo que obtenemos con Euler contra la solución exacta (analítica).

In [ ]:
# Graficamos ambas soluciones para poder compararlas
plot_euler(t, xEuler, xAna)

### Ejemplo de crecimiento poblacional

Euler también aplicó su método para estudiar cómo crece una población.

Imaginemos una ciudad, o un cultivo de bacterias, donde la cantidad de individuos crece a una tasa proporcional al número que ya existe. La ecuación es:

$$ \dfrac{dx}{dt} = \dot{x} = r x$$

Aquí, $r$ es la tasa de crecimiento, nos indica la dinámica de la población. Ahora, ¿cómo podemos observar como crece la población en el tiempo y contestar, por ejempo, que valor toma la población en un instante de tiempo dado?

Vamos a ver cómo cambia la población con el tiempo usando el método de Euler.

In [ ]:
# Número inicial de células
x0 = 1

# Valor de salto en el tiempo
dt = 0.1

# Tasa de crecimiento poblacional de células por minuto
r = 0.03

# Tiempo total de la integración en minutos
tmax = 150

# Definimos la funcion xdot
def xdot(x, t):
    return x * r

# Periodo de tiempo que queremos observar
t = np.arange(0, tmax, dt) 

# Solución analítica
xAna = x0 * np.exp(t * r)

# Solución discreta
x = np.array([x0])
for step in t:
    x = np.append(x, x[-1] + dt * xdot(x[-1], step))
xEuler = x[:-1]

plot_euler(t, xEuler, xAna)

## Usando `odeint`

El método de Euler es simple, pero no siempre muy preciso: depende mucho del tamaño de paso ($\Delta t$). Por suerte, existen funciones en Python que hacen este trabajo mejor.

Una de ellas es **`odeint`**, que viene en la librería `scipy`. Esta función nos permite resolver ecuaciones diferenciales de manera más automática y precisa.

In [ ]:
# Condición inicial de x(t) en t=0, es decir, valor de x(0).
x0 = 1

# Valor de salto en el tiempo
dt = 0.5

# Constante temporal tau
tau = 1

# Tiempo máximo que queremos integrar
tmax = 3

# Definimos la funcion xdot
def xdot(x, t):
    return x / tau

# Periodo de tiempo que queremos observar
t = np.arange(0, tmax, dt) 

# Solución analítica
xAna = x0 * np.exp(t / tau)

# Solución discreta
x = np.array([x0])
for step in t:
    x = np.append(x, x[-1] + dt * xdot(x[-1], step))
xEuler = x[:-1]

# Solución discreta con ODEint
xODEint = np.squeeze(odeint(xdot, x0, t))

# Creamos una fila con espacio para dos gráficas
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, layout='constrained')

# Graficamos las tres soluciones para poder compararlas
ax1.plot(t, xAna, "b")
ax1.plot(t, xEuler, "r--")
ax1.plot(t, xODEint, "y--")

# Graficamos el error a la derecha
ax2.plot(t, (xAna - xODEint) / xAna)

# Anotamos los colores
ax1.legend(['Solución analítica', 'Método de Euler', 'ODEint'])
ax2.legend(['Error de ODEint'])

# Anotamos los ejes
ax1.set_xlabel("t")
ax1.set_ylabel("x")
ax2.set_xlabel("t")
ax2.set_ylabel("error")

plt.show()

## Bonificación 1: Ejercicio adicional

Queremos resolver la siguiente función:

$$ f(x, t) = 1 - x + t $$

Su solución exacta es $t + e^{-t}$. Probá resolverla con Euler y compará con la solución analítica.

## Bonificación 2: Explicación del método de Euler

El cambio de una variable $x$ a lo largo de un período de tiempo entre $t$ y $t + \Delta t$ puede describirse como:

$$\Delta x = x(t + \Delta t) - x(t)$$

Por otro lado, podemos también definir el problema con una ecuación diferencial que describa el cambio de una variable $dx$ en un salto de tiempo infinitesimal $dt$. Esta dinámica contínua puede ser expresada por una función $f(x, t)$ de forma tal que:

$$f(x, t) = \dfrac{dx}{dt} = \dot{x}$$

Si en lugar de tratar el tiempo como algo continuo, lo vemos como una serie de valores discretos que cambian en pasos muy pequeños. De esta forma, podemos aproximar $\Delta x$ a $dx$ y $\Delta t$ a $dt$:

$$dx = \Delta x$$

$$dy = \Delta t$$

Con esto, las derivadas se convierten en diferencias y las integrales en sumas. Reemplazando estas nuevas equivalencias en las ecuaciónes que ya vimos, nos quedaría:

$$f(x, t) = \dfrac{\Delta x}{\Delta t} = \dfrac{x(t + \Delta t) - x(t)}{\Delta t}$$

Si despejamos $x(t + \Delta t)$ obtenemos, finalmente, una forma de calcular $x$ dado para un tiempo $t$ en forma numérica:

$$x(t + \Delta t) = x(t) + \Delta t f(x(t), t)$$